In [1]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
OPENAI_API_KEY = ""

In [3]:
prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")

prompt2 = ChatPromptTemplate.from_template("what country is the city {city} in? respond in {language}")

In [4]:
prompt1

ChatPromptTemplate(input_variables=['person'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['person'], template='what is the city {person} is from?'))])

In [5]:
prompt2

ChatPromptTemplate(input_variables=['city', 'language'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['city', 'language'], template='what country is the city {city} in? respond in {language}'))])

In [7]:
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [8]:
chain1 = prompt1 | model | StrOutputParser()

In [10]:
chain2 = (
    {'city':chain1,'language':itemgetter('language')}
    |
    prompt2
    |
    model
    |
    StrOutputParser()
)

In [12]:
chain2.invoke({"person":'trump','language':'telugu'})

'డొనాల్డ్ ట్రంప్ నగరం యొక్క దేశము అమెరికా ఉంది.'

In [15]:
from langchain_core.runnables import RunnablePassthrough

In [16]:
prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)


In [18]:
model_parser = model | StrOutputParser()

In [19]:
# color_generator = (
#     {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
# )
# color_to_fruit = prompt2 | model_parser
# color_to_country = prompt3 | model_parser
# question_generator = (
#     color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
# )
# question_generator.invoke("warm")
# prompt = question_generator.invoke("warm")
# model.invoke(prompt)

AIMessage(content='Orange is the color orange. The flag of Bhutan has an orange background with a white dragon in the middle.', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 20, 'total_tokens': 43}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [20]:
color_generator = (
    {"attribute":RunnablePassthrough()} | prompt1 | {'color':model_parser}
)
color_to_fruit =  prompt2 | model_parser
color_to_country = prompt3 | model_parser

question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)
question_generator.invoke("warm")

ChatPromptValue(messages=[HumanMessage(content='What is the color of Apple and the flag of Nepal?')])

In [21]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

AIMessage(content='Orange is the color orange, and the flag of Cyprus is white with a copper-colored silhouette of the island in the center.', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 19, 'total_tokens': 44}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [23]:
# from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
import openai

In [24]:
OPENAI_API_KEY = ""

In [26]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [27]:
synopsis_prompt = PromptTemplate.from_template(
    """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
    Title: {title}
    Playwright: This is a synopsis for the above play:"""
)

In [28]:
review_prompt = PromptTemplate.from_template(
    """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.
    Play Synopsis: {synopsis}
    Review from a New York Times play critic of the above play:"""
)

In [29]:
summary_prompt = PromptTemplate.from_template(
    """You are a play critic from the New York Times. Given the Review of play, it is your job to write a 2 sentence summary of that play.
    Play Review: {review}
    Two sentence summary of the play:"""
)

In [30]:
title = "The Tragedy of Father with no Son"

In [31]:
synopsis_chain = synopsis_prompt | llm | StrOutputParser()
review_chain = review_prompt | llm | StrOutputParser()
summary_chain = summary_prompt | llm | StrOutputParser()

In [34]:
chain = (
    { "synopsis" : synopsis_chain }
    | RunnablePassthrough.assign(review = review_chain)
    | RunnablePassthrough.assign(summary = summary_chain)
)

In [35]:
result = chain.invoke({"title": title})

In [36]:
result 

{'synopsis': 'The Tragedy of Father with no Son follows the story of a father who longs for a son to carry on his legacy, but is unable to have one. As he struggles with feelings of inadequacy and despair, he becomes consumed by his desire for a son, leading to strained relationships with those around him. The play explores themes of parenthood, legacy, and the lengths one will go to fulfill their desires. Ultimately, the father must come to terms with his situation and find a way to move forward, even without a son to call his own.',
 'review': 'In "The Tragedy of Father with no Son," playwrights have crafted a poignant and thought-provoking exploration of the complexities of parenthood and legacy. The play delves into the deep-rooted desires and struggles of a father who yearns for a son to carry on his name, only to be met with heartbreak and disappointment. As the father grapples with feelings of inadequacy and despair, the audience is taken on a journey of emotional turmoil and in

In [ ]:
chain = (
    { "synopsis" : synopsis_chain }
    | RunnablePassthrough.assign(review = review_chain)
    | RunnablePassthrough.assign(summary = summary_chain)
)